필요한 라이브러리 불러오기

In [1]:
import torch
import torch.nn as nn #G, D architecture 정의

import os # python 빌트인 os 라이브러리 import 및 파일 위치 확인
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

%matplotlib inline
import numpy as np  #행렬처리 파이썬 라이브러리
import tensorflow as tf

import matplotlib.pyplot as plt #matlab
from warnings import filterwarnings
filterwarnings('ignore')

학습 데이터셋 불러오기

* pdf

In [2]:
df = pd.read_csv('0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__ameacdneu.pdf_.csv')
print(df)

     AcroForm  AcroForm.1  DA  DR  Encoding  PDFDocEncoding  PDFDocEncoding.1  \
0           0           0   0   0         0               0                 0   
1           0           0   1   1         0               0                 0   
2           0           0   0   0         0               0                 0   
3           0           0   0   0         1               0                 0   
4           0           0   0   0         0               1                 1   
5           0           0   0   0         0               0                 0   
6           0           0   0   0         0               0                 0   
7           0           0   0   0         0               0                 0   
8           0           0   0   0         0               0                 0   
9           0           0   0   0         0               0                 0   
10          0           0   0   0         0               0                 0   
11          0           0   

In [3]:
file_path = '/home/eunbinlee/graph_data'

In [4]:
def extract_label(file_path):
    filename, _ = os.path.splitext(os.path.basename(file_path))
    MorB, etc = filename.split('_._')
    _, label = etc.split('_sample__')
    
    return np.array([MorB], dtype=np.uint16)
    #여기서 dtype을 바꿔야하나?

In [5]:
print(os.path.abspath('0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__05co1120sfige.pdf.csv'))

/home/eunbinlee/0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__05co1120sfige.pdf.csv


In [6]:
print(os.getcwd())

/home/eunbinlee


In [7]:
path = '/home/eunbinlee/graph_data/'

#file_label에 파일명(label) 리스트 추가
file_list = os.listdir(path)
file_labels = [file for file in file_list if file.startswith('0')]

print(file_labels)

['0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__14granick_WAreply.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__08inccsnap.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__6-nirmita-narasimhan.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__08rppsbryant.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__15-keith-kupferschmid.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__06cofigfs.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__13montoro.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__17_sec_sop.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__5-dan-pescod.pdf.csv', '0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__05co1120sfige.pdf.csv']


In [8]:
#위 리스트에서 앞 3개의 파일만 가지고 옴
#file_label에 대한 리스트 만들었으니 이젠 file 자체에 대한 2차원 array를 만들어보자
files = []

data1 = np.loadtxt('/home/eunbinlee/graph_data/0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__14granick_WAreply.pdf.csv', skiprows=1, delimiter=",", dtype='int32')
data2 = np.loadtxt('/home/eunbinlee/graph_data/0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__08inccsnap.pdf.csv', skiprows=1, delimiter=",", dtype='int32')
data3 = np.loadtxt('/home/eunbinlee/graph_data/0_.__cache-ben__home__cuckoo__gptest__dataset__benign_sample__6-nirmita-narasimhan.pdf.csv', skiprows=1, delimiter=",", dtype='int32')

print(len(data1[0]))
print(len(data2[0]))
print(len(data3[0]))

print(type(data1))

#file = pd.read_csv(file for file in file_list if , delimiter=',')

358
413
193
<class 'numpy.ndarray'>


### 서로 다른 길이의 데이터들 이어붙이는 keras 함수 : pad_sequences

위에서 가장 길이가 긴 file의 row 길이가 413이라는 결과를 얻었으니 모든 길이를 413으로 맞춰주자.

In [14]:
#https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

data11 = pad_sequences(data1, maxlen=413, value=-1)
data22 = pad_sequences(data2, maxlen=413, value=-1)
data33 = pad_sequences(data3, maxlen=413, value=-1)

print(data11)
print(data22)
print(data33)

[[-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 ...
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 ...
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]
 [-1 -1 -1 ...  0  0  0]]


* 지금까지 한 작업은? 
각 array들의 column의 길이를 413으로 맞춰서 -1로 padding 해주었음. 즉 가로 길이를 맞춰줬다.
지금은 얘네 다 따로 array 만들어서 padding 해줬는데, 이 array들을 unique한 column 기준으로 합한 다음에 패딩해주는 작업이 필요.

* 그 다음엔?
termination point는 어떻게 설정해주는가 고민해보자

In [ ]:
# import glob
# import natsort

# file_list = sorted(glob.glob('/graph_data/*.csv'))
# file_list = natsort.natsorted(file_list, reverse=False)

# print(file_list)

# #file 크기가 각각 달라서 해결해야 함
# #file의 max 길이 구하는 함수 찾아서 max length 넣기
# file = np.empty((len(file_list), 9999, 9999), dtype=np.uint16)
# labels = np.empty(len(file_list), dtype=np.uint16)

# for i, file_list in enumerate(file_list):
#   file = pd.read_csv(file_path, engine='python')
#   #Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.
#   file[i] = file
#   labels[i] = extract_label(file_path)


# print(len(file_list))
# #variable length to input level 
# #keras에서 제공하는 함수 사용해보기
# #자동으로 길이가 다른 파일들에 대해서 처리 (termination point를 설정해줘야할 것)

# #혜령이 아이디어) 
# #첫번째 row 읽어보고 유니크하게 만들어보고 빈 곳은 -1로 채워보는 방법

# #column을 먼저 맞추고 나서 row에 대한 처리는 keras로 해보기

In [ ]:
# pip install natsort